In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
from decimal import Decimal
import pandas as pd
import os
import urllib.request
from time import sleep
from random import randint
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np
from distutils.dir_util import remove_tree
from distutils.dir_util import copy_tree

In [2]:
mpgs = pd.read_csv('data/mpg.csv', usecols = ['make','model','year','city08','comb08','highway08','trany','drive'])

In [3]:
pics = 1 # 1 for yes
distance = 35 # miles from zip
zipcode = 98402
minp = 2500 # min price
maxp = 10000 # max price
miny = 2005 # min auto year
maxo = 150000 # max odometer reading

In [4]:
pics = "hasPic="+str(pics)+"&"
distance = "search_distance="+str(distance)+"&"
zipcode = "postal="+str(zipcode)+"&"
minp = "min_price="+str(minp)+"&"
maxp = "max_price="+str(maxp)+"&"
miny = "min_auto_year="+str(miny)+"&"
maxo = "max_auto_miles="+str(maxo)+"&"

conds = "condition=10&condition=20&condition=30&condition=40&" # exclude fair condition and below
fuel = "auto_fuel_type=1&auto_fuel_type=3&auto_fuel_type=4&" # gas, hybrid, and electric
title = "auto_title_status=1&" # clean title
tran = "auto_transmission=2&" # automatic transmission
body = "auto_bodytype=3&auto_bodytype=4&auto_bodytype=8&auto_bodytype=10&auto_bodytype=11&" # sedan, coupe, wagon, suv, hatchback

search_str = "https://seattle.craigslist.org/search/sso?"+pics+distance+zipcode+minp+maxp+miny+maxo+conds+fuel+title+tran+body
print(search_str)

https://seattle.craigslist.org/search/sso?hasPic=1&search_distance=35&postal=98402&min_price=2500&max_price=10000&min_auto_year=2005&max_auto_miles=150000&condition=10&condition=20&condition=30&condition=40&auto_fuel_type=1&auto_fuel_type=3&auto_fuel_type=4&auto_title_status=1&auto_transmission=2&auto_bodytype=3&auto_bodytype=4&auto_bodytype=8&auto_bodytype=10&auto_bodytype=11&


In [5]:
search_response = get(search_str)
search_soup = BeautifulSoup(search_response.text, 'html.parser')
results_n = search_soup.find('div', class_= 'search-legend')
N = int(results_n.find('span', class_='totalcount').text)
print("Found "+str(N)+" posts to scrape")

Found 527 posts to scrape


In [6]:
d = pd.DataFrame(columns=['url',
                          'post_name',
                          'year',
                          'make_mod',
                          'make',
                          'mod',
                          'price',
                          'location',
                          'vin',
                          'condition',
                          'drive',
                          'fuel',
                          'odometer',
                          'paint',
                          'title',
                          'transmission',
                          'type',
                          'size',
                          'cylinders',
                          'city_mpg',
                          'hwy_mpg',
                          'comb_mpg',
                          'description'])

pages = np.arange(0, N+1, 120)
iterations = 0
u = 0

for page in pages:
    
    sleep(randint(1,5))
    search = search_str + "s=" + str(page)
    search_response = get(search)
    search_soup = BeautifulSoup(search_response.text, 'html.parser')
    posts = search_soup.find_all('li', class_= 'result-row')
    
    for post in range(0,len(posts)):
        
        sleep(randint(1,5))
        url = posts[post].find('a', class_='result-title hdrlnk')['href']
        response = get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        if soup.find(id="titletextonly") is not None:
            
            d.loc[u,'url'] = url
            d.loc[u,'post_name'] = soup.find(id="titletextonly").text.strip()
            d.loc[u,'year'] = int(soup.find(class_='attrgroup').b.text[:4].strip())
            d.loc[u,'make_mod'] = soup.find(class_='attrgroup').b.text[5:].strip().lower()
            d.loc[u,'make'] = d.loc[u,'make_mod'].split(' ')[0].lower()
            if len(d.loc[u,'make_mod'].split(' ')) > 1:
                d.loc[u,'mod'] = d.loc[u,'make_mod'].split(' ')[1].lower()
            money = soup.find(class_="price").text.strip()
            d.loc[u,'price'] = Decimal(re.sub(r'[^\d.]', '', money))
            d.loc[u, 'location'] = re.sub('\W+',' ', soup.small.text.strip())

            for span_tag in soup.find_all(class_='attrgroup')[1].find_all('span'):
                if span_tag.text.split(':')[0]=='VIN':
                    d.loc[u, 'vin'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='condition':
                    d.loc[u, 'condition'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='drive':
                    d.loc[u, 'drive'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='fuel':
                    d.loc[u, 'fuel'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='odometer':
                    d.loc[u, 'odometer'] = Decimal(span_tag.text.split(':')[1].strip())
                elif span_tag.text.split(':')[0]=='paint color':
                    d.loc[u, 'paint'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='title status':
                    d.loc[u, 'title'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='transmission':
                    d.loc[u, 'transmission'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='type':
                    d.loc[u, 'type'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='size':
                    d.loc[u, 'size'] = span_tag.text.split(':')[1].strip()
                elif span_tag.text.split(':')[0]=='cylinders':
                    d.loc[u, 'cylinders'] = span_tag.text.split(':')[1].strip()

            if not mpgs[(mpgs.year == d.loc[u,'year']) &
                    (mpgs.make.str.lower() == d.loc[u,'make']) &
                    (mpgs.model.str.split().str.get(0).str.lower() == d.loc[u,'mod']) &
                    (mpgs.trany.str.split().str.get(0).str.lower() == d.loc[u,'transmission'])
                        ].empty:
                d_mpg = mpgs[(mpgs.year == d.loc[u,'year']) &
                        (mpgs.make.str.lower() == d.loc[u,'make']) &
                        (mpgs.model.str.split().str.get(0).str.lower() == d.loc[u,'mod']) &
                        (mpgs.trany.str.split().str.get(0).str.lower() == d.loc[u,'transmission'])
                            ].iloc[0]
                d.loc[u, 'city_mpg'] = int(d_mpg.city08)
                d.loc[u, 'hwy_mpg'] = int(d_mpg.highway08)
                d.loc[u, 'comb_mpg'] = int(d_mpg.comb08)

            d.loc[u, 'description'] = soup.find(id='postingbody').text[30:]
            imgUrl = soup.find(class_="slide first visible").img['src']
            local_path = os.path.join(os.getcwd(), 'imgs', str(u)+'.jpg')
            urllib.request.urlretrieve(imgUrl, local_path)
        
        u += 1
        
    iterations += 1
    print("Page " + str(iterations) + " scraped!")

d.description = d.description.str.replace('\n', '\\')

print("\n")
print("Done!")  

Page 1 scraped!
Page 2 scraped!
Page 3 scraped!
Page 4 scraped!
Page 5 scraped!


Done!


In [7]:
d

,url,post_name,year,make_mod,make,mod,price,location,vin,condition,...,paint,title,transmission,type,size,cylinders,city_mpg,hwy_mpg,comb_mpg,description
0,https://seattle.craigslist.org/see/cto/d/seatt...,2008 Nissan Altima 2.5S Sedan 4D,2008,nissan altima,nissan,altima,6500,Seattle,NaN,good,...,black,clean,automatic,sedan,full-size,4 cylinders,23,31,26,My wife and I have just relocated from Califor...
1,https://seattle.craigslist.org/see/cto/d/seatt...,2012 Honda Insight,2012,honda insight,honda,insight,7250,South Seattle,JHMZE2H38CS006218,excellent,...,red,clean,automatic,hatchback,compact,4 cylinders,40,42,41,"82750 miles, 40+mpg, one owner, clean title, d..."
2,https://seattle.craigslist.org/skc/cto/d/aubur...,2009 Lincoln MKX Pristine Condition!!,2009,lincoln mkx,lincoln,mkx,9995,google map,NaN,excellent,...,white,clean,automatic,SUV,mid-size,NaN,17,24,19,Well Apportioned Offering Sure to Delight!!\\A...
3,https://seattle.craigslist.org/see/cto/d/seatt...,2008 Honda Pilot EX-L AWD Fully Loaded Only 136K,2008,honda pilot ex-l,honda,pilot,9700,Seattle Green Lake area,5FNYF18698B029638,excellent,...,grey,clean,automatic,SUV,full-size,6 cylinders,16,22,18,Well kept 2008 Honda Pilot EX-L AWD with DVD. ...
4,https://seattle.craigslist.org/tac/cto/d/tacom...,Scion IQ,2012,scion iq,scion,iq,4000,Central Tacoma,NaN,good,...,grey,clean,automatic,hatchback,compact,4 cylinders,36,37,37,2012 Scion IQ in good condition. There is some...
5,https://seattle.craigslist.org/oly/cto/d/olymp...,2013 Hyundai Elantra GT,2013,elantra gt,elantra,gt,8500,Olympia,KMHD35LE2DUO40202,excellent,...,red,clean,automatic,hatchback,compact,4 cylinders,NaN,NaN,NaN,"One Owner, Always Garaged, Excellent Condition..."
6,https://seattle.craigslist.org/tac/cto/d/bonne...,2.5i Limited Outback Subaru 2008,2008,outback subaru,outback,subaru,7400,Bonney Lake,4S4BP62C387357279,excellent,...,silver,clean,automatic,hatchback,NaN,NaN,NaN,NaN,NaN,This is a great family car and I kind of hate ...
7,https://seattle.craigslist.org/est/cto/d/belle...,2008 Cadillac STS Black/ Black,2008,cadillac sts,cadillac,sts,4500,Bellevue,NaN,excellent,...,black,clean,automatic,sedan,full-size,6 cylinders,17,26,20,We bought this car brand new and always had it...
8,https://seattle.craigslist.org/tac/cto/d/tacom...,2005 Acura RL 50k original miles,2005,acura rl,acura,rl,10000,Puyallup,NaN,excellent,...,grey,clean,automatic,sedan,full-size,6 cylinders,16,24,19,We have decided to sell our car as we just bou...
9,https://seattle.craigslist.org/see/cto/d/seatt...,2010 Toyota Carolla,2010,toyota carrola le,toyota,carrola,6500,Seattle Seward Park,2T1BU4EE8AC438412,good,...,white,clean,automatic,sedan,mid-size,4 cylinders,NaN,NaN,NaN,2010 Toyota Carolla LE. Reliable; 4-door sedan...


In [8]:
d.to_csv(r'data/cars.csv', index=True, header=True)

In [10]:
# rename the photos to match the sorting convention of Tableau custom shapes

photo_is = d.index.values.tolist()
photo_is = [str(i) for i in photo_is]
p = sorted(photo_is)

for filename in os.listdir('imgs'):
    if not filename.startswith('.'):
        num = int(re.findall(r'^\d+', filename)[0])
        lookup = p[num]        
        temp_dst = "t" + str(lookup) + ".jpg"
        src = os.path.join(os.getcwd(), 'imgs', filename)
        dst = os.path.join(os.getcwd(), 'imgs', temp_dst)
        os.rename(src, dst)
        
for f in os.listdir('imgs'): # have to rename in two steps so os.rename doesn't delete files
    if not f.startswith('.'):        
        src = os.path.join(os.getcwd(), 'imgs', f)
        dst = os.path.join(os.getcwd(), 'imgs', f[1:])
        os.rename(src, dst)

In [11]:
src = os.path.join(os.getcwd(), 'imgs')
dst = '/Users/Brobot/Documents/My Tableau Repository/Shapes/Cars'

remove_tree(dst)
copy_tree(src, dst)

['/Users/Brobot/Documents/My Tableau Repository/Shapes/Cars/63.jpg',
 '/Users/Brobot/Documents/My Tableau Repository/Shapes/Cars/189.jpg',
 '/Users/Brobot/Documents/My Tableau Repository/Shapes/Cars/77.jpg',
 '/Users/Brobot/Documents/My Tableau Repository/Shapes/Cars/162.jpg',
 '/Users/Brobot/Documents/My Tableau Repository/Shapes/Cars/176.jpg',
 '/Users/Brobot/Documents/My Tableau Repository/Shapes/Cars/88.jpg',
 '/Users/Brobot/Documents/My Tableau Repository/Shapes/Cars/348.jpg',
 '/Users/Brobot/Documents/My Tableau Repository/Shapes/Cars/360.jpg',
 '/Users/Brobot/Documents/My Tableau Repository/Shapes/Cars/406.jpg',
 '/Users/Brobot/Documents/My Tableau Repository/Shapes/Cars/412.jpg',
 '/Users/Brobot/Documents/My Tableau Repository/Shapes/Cars/374.jpg',
 '/Users/Brobot/Documents/My Tableau Repository/Shapes/Cars/228.jpg',
 '/Users/Brobot/Documents/My Tableau Repository/Shapes/Cars/214.jpg',
 '/Users/Brobot/Documents/My Tableau Repository/Shapes/Cars/200.jpg',
 '/Users/Brobot/Documen